In [116]:
import pandas as pd
import requests
import osmnx as ox
import numpy as np

In [117]:
# content = requests.get("https://docs.google.com/spreadsheets/d/1KLAUoLS5GV03vDsQLrItBQb4IEnQfC9ZHgE6coDZJkQ/export?format=csv").content
url_tps = "https://docs.google.com/spreadsheets/d/1KLAUoLS5GV03vDsQLrItBQb4IEnQfC9ZHgE6coDZJkQ/export?format=csv"
df_tps = pd.read_csv(url_tps)
url_kecamatan = "https://docs.google.com/spreadsheets/d/1fTwujwqyiMhE_VNw_CNCQGdxcbBgNpAcTeJDcSW5TpM/export?format=csv"
df_kecamatan = pd.read_csv(url_kecamatan)
df_tps.head(5)
# df_kecamatan.head(5)

,Koordinat,Jenis,Nama Fasilitas,Kapasitas
0,"-6.9335863,107.698076",TPST/TPS 3R,TPST Gedebage,NaN
1,"-6.936889,107.568306",TPST/TPS 3R,TPST Cicukang Holis 2,NaN
2,"-6.931722,107.599722",TPST/TPS 3R,TPST Nyengseret,NaN
3,"-6.932917,107.606472",TPST/TPS 3R,TPST Tegalega,NaN
4,"-6.8851639,107.6088417",TPST/TPS 3R,TPST Babakan Siliwangi,NaN


In [118]:
df_tps.columns = df_tps.columns.str.lower()
df_tps.columns = df_tps.columns.str.replace(" ", "_")
df_tps[["lat", "long"]] = df_tps["koordinat"].str.split(",", expand=True)
df_tps = df_tps.rename(columns={"nama_fasilitas": "nama"})

df_kecamatan.columns = df_kecamatan.columns.str.lower()
df_kecamatan.columns = df_kecamatan.columns.str.replace(" ", "_")
df_kecamatan = df_kecamatan.rename(columns={"kota_dan_kecamatan": "nama", "semester_i": "semester_1", "semester_ii": "semester_2"})

df_tps.head(5)
df_kecamatan.head(5)

,nama,semester_1,semester_2
0,Bandung Kulon,138609,138680
1,Babakan Ciparay,147541,148721
2,Bojongloa Kaler,125950,126385
3,Bojongloa Kidul,89833,90030
4,Astanaanyar,73794,73853


In [119]:
df_tps["kapasitas"] = 9

df_kecamatan["sampah_ton_hari"] = df_kecamatan["semester_2"] * 0.6 / 1000
df_kecamatan = df_kecamatan.drop(30)

df_kecamatan
# print(df_kecamatan["sampah_ton_hari"].sum())

,nama,semester_1,semester_2,sampah_ton_hari
0,Bandung Kulon,138609,138680,83.2080
1,Babakan Ciparay,147541,148721,89.2326
2,Bojongloa Kaler,125950,126385,75.8310
3,Bojongloa Kidul,89833,90030,54.0180
4,Astanaanyar,73794,73853,44.3118
5,Regol,81580,81706,49.0236
6,Lengkong,71966,71906,43.1436
7,Bandung Kidul,62666,63083,37.8498
8,Buahbatu,107327,108064,64.8384
9,Rancasari,89042,89756,53.8536


In [120]:
# 1. Siapkan kolom kosong terlebih dahulu
df_kecamatan["lat"] = None
df_kecamatan["long"] = None

# 2. Loop berdasarkan Index
for index in df_kecamatan.index:
    # Ambil nama kecamatan dari row saat ini
    nama_kecamatan = df_kecamatan.at[index, "nama"]
    
    # Buat query pencarian lengkap
    search_query = f"{nama_kecamatan}, Kota Bandung, Indonesia"
    
    print(f"Sedang memproses: {search_query}...")
    
    try:
        # --- PROSES OSMNX ---
        # Ambil boundary
        gdf = ox.geocode_to_gdf(search_query)

        # 3. Hitung Centroid
        # Hasilnya adalah objek Point (longitude, latitude)
        centroid = gdf.geometry.centroid
        
        # Ambil nilai
        lat_center = centroid.y.iloc[0]
        long_center = centroid.x.iloc[0]
        
        # --- CARA INPUT KE DATAFRAME ---
        # Gunakan .at[index, nama_kolom]
        df_kecamatan.at[index, "lat"] = lat_center
        df_kecamatan.at[index, "long"] = long_center
        
    except Exception as e:
        # Jika kecamatan tidak ditemukan atau error koneksi
        print(f"Gagal mengambil data untuk {search_query}. Error: {e}")
        # Nilai akan tetap None (kosong)

# Cek hasil akhir
print(df_kecamatan.head())

Sedang memproses: Bandung Kulon, Kota Bandung, Indonesia...
Sedang memproses: Babakan Ciparay, Kota Bandung, Indonesia...
Sedang memproses: Bojongloa Kaler, Kota Bandung, Indonesia...
Sedang memproses: Bojongloa Kidul, Kota Bandung, Indonesia...
Sedang memproses: Astanaanyar, Kota Bandung, Indonesia...
Sedang memproses: Regol, Kota Bandung, Indonesia...
Sedang memproses: Lengkong, Kota Bandung, Indonesia...
Sedang memproses: Bandung Kidul, Kota Bandung, Indonesia...
Sedang memproses: Buahbatu, Kota Bandung, Indonesia...
Sedang memproses: Rancasari, Kota Bandung, Indonesia...
Sedang memproses: Gedebage, Kota Bandung, Indonesia...
Sedang memproses: Cibiru, Kota Bandung, Indonesia...
Sedang memproses: Panyileukan, Kota Bandung, Indonesia...
Sedang memproses: Ujungberung, Kota Bandung, Indonesia...
Sedang memproses: Cinambo, Kota Bandung, Indonesia...
Sedang memproses: Arcamanik, Kota Bandung, Indonesia...
Sedang memproses: Antapani, Kota Bandung, Indonesia...
Sedang memproses: Mandalajati

C:\Users\acer\AppData\Local\Temp\ipykernel_17820\2473424296.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid
C:\Users\acer\AppData\Local\Temp\ipykernel_17820\2473424296.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid
C:\Users\acer\AppData\Local\Temp\ipykernel_17820\2473424296.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid
C:\Users\acer\AppData\Local\Temp\ipykernel_17820\2473424296.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' ar

Sedang memproses: Kiaracondong, Kota Bandung, Indonesia...
Sedang memproses: Batununggal, Kota Bandung, Indonesia...
Sedang memproses: Sumur Bandung, Kota Bandung, Indonesia...
Sedang memproses: Andir, Kota Bandung, Indonesia...
Sedang memproses: Cicendo, Kota Bandung, Indonesia...
Sedang memproses: Bandung Wetan, Kota Bandung, Indonesia...
Sedang memproses: Cibeunying Kidul, Kota Bandung, Indonesia...
Sedang memproses: Cibeunying Kaler, Kota Bandung, Indonesia...
Sedang memproses: Coblong, Kota Bandung, Indonesia...
Sedang memproses: Sukajadi, Kota Bandung, Indonesia...
Sedang memproses: Sukasari, Kota Bandung, Indonesia...
Sedang memproses: Cidadap, Kota Bandung, Indonesia...
              nama  semester_1  semester_2  sampah_ton_hari       lat  \
0    Bandung Kulon      138609      138680          83.2080 -6.931001   
1  Babakan Ciparay      147541      148721          89.2326 -6.944026   
2  Bojongloa Kaler      125950      126385          75.8310 -6.932856   
3  Bojongloa Kidul   

C:\Users\acer\AppData\Local\Temp\ipykernel_17820\2473424296.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid
C:\Users\acer\AppData\Local\Temp\ipykernel_17820\2473424296.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid
C:\Users\acer\AppData\Local\Temp\ipykernel_17820\2473424296.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid
C:\Users\acer\AppData\Local\Temp\ipykernel_17820\2473424296.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' ar

In [121]:
df_tpa = pd.DataFrame()

gdf = ox.geocode_to_gdf("TPA Sarimukti, Sarimukti, Kabupaten Bandung Barat, Indonesia")

# 3. Hitung Centroid
# Hasilnya adalah objek Point (longitude, latitude)
centroid = gdf.geometry.centroid

# Ambil nilai
lat_center = centroid.y.iloc[0]
long_center = centroid.x.iloc[0]

df_tpa["nama"] = ["TPA Sarimukti"]
df_tpa["lat"] = [lat_center]
df_tpa["long"] = [long_center]

C:\Users\acer\AppData\Local\Temp\ipykernel_17820\3755340377.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = gdf.geometry.centroid


In [122]:
df_kecamatan

,nama,semester_1,semester_2,sampah_ton_hari,lat,long
0,Bandung Kulon,138609,138680,83.2080,-6.931001,107.564819
1,Babakan Ciparay,147541,148721,89.2326,-6.944026,107.578726
2,Bojongloa Kaler,125950,126385,75.8310,-6.932856,107.589448
3,Bojongloa Kidul,89833,90030,54.0180,-6.951276,107.598183
4,Astanaanyar,73794,73853,44.3118,-6.933676,107.601609
5,Regol,81580,81706,49.0236,-6.940135,107.612612
6,Lengkong,71966,71906,43.1436,-6.933838,107.624776
7,Bandung Kidul,62666,63083,37.8498,-6.95765,107.631185
8,Buahbatu,107327,108064,64.8384,-6.950172,107.655856
9,Rancasari,89042,89756,53.8536,-6.954563,107.673752


In [123]:
df_tps

,koordinat,jenis,nama,kapasitas,lat,long
0,"-6.9335863,107.698076",TPST/TPS 3R,TPST Gedebage,9,-6.9335863,107.698076
1,"-6.936889,107.568306",TPST/TPS 3R,TPST Cicukang Holis 2,9,-6.936889,107.568306
2,"-6.931722,107.599722",TPST/TPS 3R,TPST Nyengseret,9,-6.931722,107.599722
3,"-6.932917,107.606472",TPST/TPS 3R,TPST Tegalega,9,-6.932917,107.606472
4,"-6.8851639,107.6088417",TPST/TPS 3R,TPST Babakan Siliwangi,9,-6.8851639,107.6088417
...,...,...,...,...,...,...
173,"-6.8750488,107.6053536",TPST/TPS 3R,TPS 3 R Universitas Katolik Parahyangan,9,-6.8750488,107.6053536
174,"-6.8889179,107.6077555",TPST/TPS 3R,Universitas Islam Bandung,9,-6.8889179,107.6077555
175,"-6.88643329999999,107.5809448",TPST/TPS 3R,Universitas Kristen Maranatha,9,-6.88643329999999,107.5809448
176,"-6.9465473,107.6216576",TPST/TPS 3R,TPS 3 R Pasirluyu,9,-6.9465473,107.6216576


In [124]:
df_tpa

,nama,lat,long
0,TPA Sarimukti,-6.800792,107.348365


In [ ]:
# 1. Fungsi Utama OSRM Table
def get_osrm_table(sources_df, dests_df, osrm_endpoint="http://router.project-osrm.org"):
    """
    Menghitung matriks jarak dan durasi dari Sources ke Destinations.
    Input berupa DataFrame dengan kolom 'lat' dan 'long'.
    """
    
    # Gabungkan semua koordinat (Sources dulu, baru Destinations)
    # Format OSRM string: "lon1,lat1;lon2,lat2;..."
    src_coords = [f"{r['long']},{r['lat']}" for _, r in sources_df.iterrows()]
    dst_coords = [f"{r['long']},{r['lat']}" for _, r in dests_df.iterrows()]
    
    all_coords = ";".join(src_coords + dst_coords)
    
    # Tentukan index untuk parameter source dan destination
    # Misal: 3 Source, 2 Dest. List total ada 5. 
    # sources idx = 0,1,2. destinations idx = 3,4.
    n_src = len(sources_df)
    n_dst = len(dests_df)
    
    src_indices = ";".join([str(i) for i in range(n_src)])
    dst_indices = ";".join([str(i) for i in range(n_src, n_src + n_dst)])
    
    # Susun URL Request
    url = f"{osrm_endpoint}/table/v1/driving/{all_coords}"
    params = {
        "sources": src_indices,
        "destinations": dst_indices,
        "annotations": "duration,distance" # Minta waktu dan jarak
    }
    
    print(f"Requesting OSRM matrix: {n_src} origins x {n_dst} destinations...")
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        
        # Hasilnya adalah matrix (list of lists)
        durations = data['durations'] # dalam detik
        distances = data['distances'] # dalam meter
        
        # KITA PERLU MERAPIKAN HASILNYA MENJADI TABEL DATAFRAME
        results = []
        for i, src_row in enumerate(durations):
            for j, duration_val in enumerate(src_row):
                dist_val = distances[i][j]
                
                # Ambil nama/ID dari dataframe asli
                # (Asumsi ada kolom 'nama' atau 'id', sesuaikan dengan data Anda)
                src_name = sources_df.iloc[i].get('nama', f"Src_{i}")
                dst_name = dests_df.iloc[j].get('nama', f"Dst_{j}")
                
                results.append({
                    "origin": src_name,
                    "destination": dst_name,
                    "duration_minutes": duration_val / 60 if duration_val else None, # Detik ke Menit
                    "distance_km": dist_val / 1000 if dist_val else None # Meter ke KM
                })
                
        return pd.DataFrame(results)
    else:
        print("Error:", response.status_code, response.text)
        return pd.DataFrame()

# --- CONTOH PENGGUNAAN ---

# 0. Data Dummy (Sesuaikan dengan df real Anda)
# Pastikan tipe data float
# df_kecamatan = pd.DataFrame({
#     'kota_dan_kecamatan': ['Kec. Coblong', 'Kec. Sukasari'],
#     'lat': [-6.8906, -6.8756],
#     'long': [107.6186, 107.5876]
# })

# df_tps = pd.DataFrame({
#     'nama_tps': ['TPS Pasar Simpang', 'TPS Sabuga', 'TPS Gegerkalong'],
#     'lat': [-6.8920, -6.8850, -6.8700],
#     'long': [107.6160, 107.6100, 107.5900]
# })

# # TPA Sarimukti (Hanya 1 baris)
# df_tpa = pd.DataFrame({
#     'nama_tps': ['TPA Sarimukti'], # Samakan nama kolom biar fungsi reusable
#     'lat': [-6.797282], # Koordinat contoh Sarimukti
#     'long': [107.352425]
# })

--- TAHAP 1 ---
Requesting OSRM matrix: 30 origins x 178 destinations...
          origin             destination  duration_minutes  distance_km
0  Bandung Kulon           TPST Gedebage         15.975000      17.7860
1  Bandung Kulon   TPST Cicukang Holis 2          1.938333       1.0880
2  Bandung Kulon         TPST Nyengseret          7.280000       6.2959
3  Bandung Kulon           TPST Tegalega          7.816667       6.9717
4  Bandung Kulon  TPST Babakan Siliwangi         10.281667      10.4942

--- TAHAP 2 ---
Requesting OSRM matrix: 178 origins x 1 destinations...
                   origin    destination  duration_minutes  distance_km
0           TPST Gedebage  TPA Sarimukti         46.520000      54.0910
1   TPST Cicukang Holis 2  TPA Sarimukti         36.515000      40.2664
2         TPST Nyengseret  TPA Sarimukti         37.776667      42.5261
3           TPST Tegalega  TPA Sarimukti         38.551667      43.3170
4  TPST Babakan Siliwangi  TPA Sarimukti         39.518333    

In [126]:
# --- TAHAP 1: KECAMATAN KE TPS ---
print("--- TAHAP 1 ---")
df_result_kec_to_tps = get_osrm_table(df_kecamatan, df_tps)
print(df_result_kec_to_tps)


# --- TAHAP 2: TPS KE TPA SARIMUKTI ---
print("\n--- TAHAP 2 ---")
# Di sini TPS jadi Source, TPA jadi Destination
df_result_tps_to_tpa = get_osrm_table(df_tps, df_tpa)
print(df_result_tps_to_tpa)

--- TAHAP 1 ---
Requesting OSRM matrix: 30 origins x 178 destinations...
             origin                              destination  \
0     Bandung Kulon                            TPST Gedebage   
1     Bandung Kulon                    TPST Cicukang Holis 2   
2     Bandung Kulon                          TPST Nyengseret   
3     Bandung Kulon                            TPST Tegalega   
4     Bandung Kulon                   TPST Babakan Siliwangi   
...             ...                                      ...   
5335        Cidadap  TPS 3 R Universitas Katolik Parahyangan   
5336        Cidadap                Universitas Islam Bandung   
5337        Cidadap            Universitas Kristen Maranatha   
5338        Cidadap                        TPS 3 R Pasirluyu   
5339        Cidadap                        TPS 3 R Ence Azis   

      duration_minutes  distance_km  
0            15.975000      17.7860  
1             1.938333       1.0880  
2             7.280000       6.2959  
3     